In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('hotel_bookings_train_data.csv', sep = ";")

print(data[:1])

   booking_id         hotel  is_canceled  lead_time  arrival_date_year  \
0           0  Resort Hotel            0        342               2015   

  arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  \
0               July                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  ...  agent  \
0                        0                     0  ...    0.0   

   days_in_waiting_list  customer_type  adr required_car_parking_spaces  \
0                     0      Transient  0.0                           0   

  total_of_special_requests reservation_status  reservation_status_date  \
0                         0          Check-Out               01/07/2015   

   arrival_date  booking_date  
0    01/07/2015    24/07/2014  

[1 rows x 34 columns]


In [72]:
# Convert 'booking_date' to datetime if it's not already.
data['booking_date'] = pd.to_datetime(data['booking_date'], format='%d/%m/%Y')

hotel_bookings = data[data['booking_date'] < '2017-08-01'] # Data before ingestion begins

# Filter rows where 'booking_date' is August 1st, 2017.
data_profiling_before = data[data['booking_date'] == '2017-08-01'] # First day of ingestion. This will need to be automated with an Airflow variable.

# Dropping the 'is_canceled' column from the filtered data.
data_to_predict = data_profiling_before.drop(columns=['is_canceled'])

# Saving the filtered and cleaned data to a CSV file.
file_path = 'data_to_predict.csv'
data_to_predict.to_csv(file_path, index=False)



In [4]:
data_profiling_before.columns

Index(['booking_id', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'arrival_date',
       'booking_date'],
      dtype='object')

In [5]:
hotel_bookings.columns

Index(['booking_id', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'arrival_date',
       'booking_date'],
      dtype='object')

In [6]:
cat_col = ['meal','country','market_segment','distribution_channel','is_repeated_guest','reserved_room_type','assigned_room_type','deposit_type','agent','customer_type',]

for col in cat_col:
    print(f"Unique values in {col}: {data[col].unique()}")
    
cat_col_eval = ['meal','market_segment','distribution_channel','is_repeated_guest','reserved_room_type','assigned_room_type','deposit_type','customer_type']


Unique values in meal: ['BB' 'FB' 'HB' 'SC' 'Undefined']
Unique values in country: ['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG' 'POL' 'DEU'
 'BEL' 'CHE' 'CN' 'GRC' 'ITA' 'NLD' 'DNK' 'RUS' 'SWE' 'AUS' 'EST' 'CZE'
 'BRA' 'FIN' 'MOZ' 'BWA' 'LUX' 'SVN' 'ALB' 'IND' 'CHN' 'MEX' 'MAR' 'UKR'
 'SMR' 'LVA' 'PRI' 'SRB' 'CHL' 'AUT' 'BLR' 'LTU' 'TUR' 'ZAF' 'AGO' 'ISR'
 'CYM' 'ZMB' 'CPV' 'ZWE' 'DZA' 'KOR' 'CRI' 'HUN' 'ARE' 'TUN' 'JAM' 'HRV'
 'HKG' 'IRN' 'GEO' 'AND' 'GIB' 'URY' 'JEY' 'CAF' 'CYP' 'COL' 'GGY' 'KWT'
 'NGA' 'MDV' 'VEN' 'SVK' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'SEN' 'SYC'
 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MKD' 'MYS' 'ARM' 'JPN' 'LKA' 'CUB' 'CMR'
 'BIH' 'MUS' 'COM' 'SUR' 'UGA' 'BGR' 'CIV' 'JOR' 'SYR' 'SGP' 'BDI' 'SAU'
 'VNM' 'PLW' 'QAT' 'EGY' 'PER' 'MLT' 'MWI' 'ECU' 'MDG' 'ISL' 'UZB' 'NPL'
 'BHS' 'MAC' 'TGO' 'TWN' 'DJI' 'STP' 'KNA' 'ETH' 'IRQ' 'HND' 'RWA' 'KHM'
 'MCO' 'BGD' 'IMN' 'TJK' 'NIC' 'BEN' 'VGB' 'TZA' 'GAB' 'GHA' 'TMP' 'GLP'
 'KEN' 'LIE' 'GNB' 'MNE' 'UMI' 'MYT' 'FRO'

## Try to put things in a Flask App

## Data Exploration

In [8]:
numerical_cols = ['lead_time','stays_in_weekend_nights','stays_in_week_nights', 'adults', 'children', 'babies','previous_cancellations','previous_bookings_not_canceled','days_in_waiting_list',  'adr','total_of_special_requests',]


In [73]:
print('Correlation Analysis')
message_correlation_analysis = 'Correlation Analysis'

print('hotel_bookings:')
title_hotel_bookings_corr = 'hotel_bookings correlation:'
correlation_matrix = hotel_bookings[numerical_cols].corr()
# Initialize the list to store messages
message_hotel_bookings_corr = []
# Iterate over the correlation matrix
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):  # i+1 to skip diagonal
        if correlation_matrix.iloc[i, j] < -0.4 or correlation_matrix.iloc[i, j] > 0.4:
            alert_message = f"The correlation between {correlation_matrix.columns[i]} and {correlation_matrix.columns[j]} is {correlation_matrix.iloc[i, j]:.2f} //"
            print(alert_message)
            # Append the message to the list
            message_hotel_bookings_corr.append(alert_message)

message_hotel_bookings_corr = "\n".join(message_hotel_bookings_corr)

print("data_profiling_before correlation:")
title_data_profiling_before = "data_profiling_before:"
correlation_matrix = data_profiling_before[numerical_cols].corr()
# Initialize the list to store messages
message_data_profiling_before = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):  # i+1 to skip diagonal
        if correlation_matrix.iloc[i, j] < -0.4 or correlation_matrix.iloc[i, j] > 0.4:
            alert_message = f"The correlation between {correlation_matrix.columns[i]} and {correlation_matrix.columns[j]} is {correlation_matrix.iloc[i, j]:.2f} //"
            print(alert_message)
            # Append the message to the list
            message_data_profiling_before.append(alert_message)

message_data_profiling_before = "\n".join(message_data_profiling_before)


Correlation Analysis
hotel_bookings:
The correlation between stays_in_weekend_nights and stays_in_week_nights is 0.50 //
data_profiling_before correlation:
The correlation between stays_in_weekend_nights and stays_in_week_nights is 0.76 //
The correlation between children and adr is 0.66 //
The correlation between previous_cancellations and previous_bookings_not_canceled is 0.94 //
The correlation between previous_bookings_not_canceled and adr is -0.43 //


In [66]:
import pandas as pd

# Assuming data_profiling_before is your DataFrame

# Count the number of nulls in each column
print("Null counts in each column:")
title_count_nulls = "Null counts in each column:"
null_counts = data_profiling_before.isnull().sum()
print(null_counts)

# Create a DataFrame with column names and null counts
message_count_nulls = pd.DataFrame({'Column Name': null_counts.index, 'Null Count': null_counts.values})

# Convert the DataFrame to an HTML table
message_count_nulls = message_count_nulls.to_html(classes='my-table', index=False, border=0)



numerical_cols = ['lead_time','stays_in_weekend_nights','stays_in_week_nights', 'adults', 'children', 'babies','previous_cancellations','previous_bookings_not_canceled','days_in_waiting_list',  'adr','total_of_special_requests',]
binary_cols = ['is_canceled','is_repeated_guest','required_car_parking_spaces',]
categorical_cols = ['hotel','meal','country', 'market_segment', 'distribution_channel','reserved_room_type','assigned_room_type', 'booking_changes', 'deposit_type', 'agent','customer_type','reservation_status',]
time_cols = ['booking_id','arrival_date_year', 'arrival_date_month', 'arrival_date_week_number','arrival_date_day_of_month','reservation_status_date', 'arrival_date', 'booking_date']
        
title_count_cat = 'Number of categorical columns'
print(f"\nCount of categorical columns: {len(categorical_cols)}")
message_categorical = f"\nCount of categorical columns: {len(categorical_cols)}"
title_count_num = 'Number of numerical columns'
print(f"Count of numerical columns: {len(numerical_cols)}")
message_numerical = f"Count of numerical columns: {len(numerical_cols)}"
title_count_bin = 'Number of binary columns'
print(f"Count of binary columns: {len(binary_cols)}") 
message_binary = f"Count of binary columns: {len(binary_cols)}"  

# Define a function to detect outliers using IQR
def detect_outliers_iqr(data):
    mean = data.mean()
    std = data.std()
    lower_bound = mean - 4 * std
    upper_bound = mean + 4 * std
    return data[(data < lower_bound) | (data > upper_bound)]

# Initialize the DataFrame to store outliers
outliers_df = pd.DataFrame()

# Detect and store outliers for each numerical column
for col in numerical_cols:
    outliers = detect_outliers_iqr(data_profiling_before[col])
    outliers_df[col] = data_profiling_before[col].isin(outliers)

# Now, create a mask that indicates whether each row has at least one outlier
outliers_mask = outliers_df.any(axis=1)

print('Check for outliers --> 4 x standard deviation ')
title_outliers = 'Check for outliers --> 4 x standard deviation '

message_outliers_numeric = []
message_outliers_binary = []
        
for index, row in outliers_df.iterrows():
    if row.any():  # Check if the row has any True values indicating outliers
        # Retrieve the booking_id for the current row
        booking_id = data_profiling_before.loc[index, 'booking_id']
        
        # Get the list of column names where this row is an outlier
        outlier_columns = row[row].index.tolist()
        
        # Retrieve the values of the outlier columns for the current row
        outlier_values = data_profiling_before.loc[index, outlier_columns].to_dict()
        
        # Create the message for this outlier
        outlier_message = f"Booking ID {booking_id} outlier values: {outlier_values} "
        print(outlier_message)
        
        # Append the message to the list
        message_outliers_numeric.append(outlier_message)

        
print('Check for non-binary values in binary columns')
for col in binary_cols:
    non_binary_rows = data_profiling_before[~data_profiling_before[col].isin([0, 1])]
    if not non_binary_rows.empty:
        for index, row in non_binary_rows.iterrows():
            booking_id = row['booking_id']
            # Create the message for this non-binary value
            non_binary_message = f"Non-binary alert for booking ID {booking_id} in column {col}: Value = {row[col]}"
            print(non_binary_message)
            
            # Append the message to the list
            message_outliers_binary.append(non_binary_message)

# Merge the lists
merged_outliers_message = message_outliers_numeric + message_outliers_binary

# Convert the merged list to a DataFrame
merged_outliers_message = pd.DataFrame(merged_outliers_message, columns=['Message'])

# Convert the DataFrame to an HTML table
merged_outliers_message = merged_outliers_message.to_html(classes='my-table', index=False, border=0)


Null counts in each column:
booking_id                        0
hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
days_in_waiting_list              0
customer_type                     0


In [13]:
# Calculate quartiles
Q1 = data_profiling_before['children'].quantile(0.25)
Q2 = data_profiling_before['children'].quantile(0.5)
Q3 = data_profiling_before['children'].quantile(0.75)

# Print quartiles
print(f"First quartile (Q1): {Q1}")
print(f"Second quartile (Q2) / Median: {Q2}")
print(f"Third quartile (Q3): {Q3}")


First quartile (Q1): 0.0
Second quartile (Q2) / Median: 0.0
Third quartile (Q3): 0.0


## Input Features Drift Detection

In [68]:
import scipy.stats as stats
import pandas as pd

print('New Data Exploration for Numerical Columns')

title_data_drift_num = 'Exploring input fature data drift for numerical columns'

print('We only study features generated at booking time. For instance, we exclude number of days in waitlist.')

message_data_drift_num = 'We only study features generated at booking time. For instance, we exclude number of days in waitlist.'

print('T-TEST AND K-S TEST')

title_ttest = 'T-TEST RESULTS. Significantly different columns'

title_kstest = 'K-S TEST RESULTS. Significantly different columns'

num_cols = [
    'lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights',
    'adults', 'children', 'babies', 'previous_cancellations',
    'previous_bookings_not_canceled', 'adr', 'total_of_special_requests'
]

# Initialize dictionaries to store the p-values for each test
p_values_ttest = {}
p_values_ks = {}

# Dictionary to store means for each group and column
means = {}

p_value_threshold = 0.01

for col in num_cols:
    # Perform the T-test
    t_stat, p_val_ttest = stats.ttest_ind(
        hotel_bookings[col].dropna(),
        data_profiling_before[col].dropna(),
        equal_var=False
    )
    # Store the p-value from the T-test in the dictionary
    p_values_ttest[col] = p_val_ttest
    
    # Perform the Kolmogorov-Smirnov test
    ks_stat, p_val_ks = stats.ks_2samp(
        hotel_bookings[col].dropna(),
        data_profiling_before[col].dropna()
    )
    # Store the p-value from the K-S test in the dictionary
    p_values_ks[col] = p_val_ks
    
    # Calculate and store the means
    mean_hotel_bookings = hotel_bookings[col].mean()
    mean_data_profiling_before = data_profiling_before[col].mean()
    means[col] = (mean_hotel_bookings, mean_data_profiling_before)

# Prepare data for DataFrame for T-test
significant_data_ttest = [
    (
        col, 
        p_values_ttest[col], 
        means[col][0],  # Old Mean (hotel_bookings)
        means[col][1],  # New Mean (data_profiling_before)
        ((means[col][1] - means[col][0]) / means[col][0] * 100) if means[col][0] != 0 else 'Infinity' # Percentage Change
    ) 
    for col in num_cols 
    if p_values_ttest[col] < p_value_threshold
]

# Convert to DataFrame with columns for p-value, old mean, new mean, and percentage change for T-test
significant_cols_df_ttest = pd.DataFrame(
    significant_data_ttest, 
    columns=['Feature', 'P-value (T-test)', 'Old Mean', 'New Mean', 'Percentage Change']
)

# Prepare data for DataFrame for K-S test
significant_data_ks = [
    (col, p_values_ks[col]) for col in num_cols if p_values_ks[col] < p_value_threshold
]

# Convert to DataFrame with columns for feature and p-value for K-S test
significant_cols_df_ks = pd.DataFrame(
    significant_data_ks, 
    columns=['Feature', 'P-value (K-S test)']
)

# Print results
if not significant_cols_df_ttest.empty:
    print('T-TEST Alert! We found significant differences in the new data based on T-test!')
    print(significant_cols_df_ttest)
    message_ttest = significant_cols_df_ttest.to_html(classes='my-table', index=False, border=0)

    print(message_ttest)
else:
    print(f"No significant differences found at the specified {p_value_threshold} threshold.")
    message_ttest = f"No significant differences found at the specified {p_value_threshold} threshold."

if not significant_cols_df_ks.empty:
    print('K-S TEST Alert! We found significant differences in the new data based on K-S test!')
    print(significant_cols_df_ks)
    message_kstest = significant_cols_df_ks.to_html(classes='my-table', index=False, border=0)
else:
    print(f"No significant differences found at the specified {p_value_threshold} threshold.")
    message_kstest = f"No significant differences found at the specified {p_value_threshold} threshold."


New Data Exploration for Numerical Columns
We only study features generated at booking time. For instance, we exclude number of days in waitlist.
T-TEST AND K-S TEST
T-TEST Alert! We found significant differences in the new data based on T-test!
                     Feature  P-value (T-test)    Old Mean    New Mean  \
0                  lead_time      1.396399e-65  104.915646   11.258065   
1                     babies     3.308886e-172    0.007938    0.000000   
2                        adr      8.444194e-11  101.573857  170.185161   
3  total_of_special_requests      7.473223e-04    0.569249    1.048387   

   Percentage Change  
0         -89.269413  
1        -100.000000  
2          67.548193  
3          84.170257  
<table class="dataframe my-table">
  <thead>
    <tr style="text-align: right;">
      <th>Feature</th>
      <th>P-value (T-test)</th>
      <th>Old Mean</th>
      <th>New Mean</th>
      <th>Percentage Change</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>l

In [70]:
print('New Data Exploration for Categorical Columns')

title_chi2 = 'CHI SQUARED RESULTS for categorical columns. Significantly different columns'
print('CHI^2 TEST')

def chi2_test(cat_col, df1, df2):
    # Calculate the frequency of each category in both dataframes
    freq_df1 = df1[cat_col].value_counts().to_frame(name='count1')
    freq_df2 = df2[cat_col].value_counts().to_frame(name='count2')
    
    # Merge the frequencies into a single dataframe
    freq_df = freq_df1.merge(freq_df2, left_index=True, right_index=True, how='outer').fillna(0)
    
    # Perform the Chi-squared test
    chi2_stat, p_val, dof, expected = stats.chi2_contingency(freq_df)
    
    return p_val

# List of categorical columns to evaluate
cat_col_eval = [
    'meal', 'market_segment', 'distribution_channel', 'is_repeated_guest',
    'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type'
]

print('New Data Exploration')

# Initialize dictionary to store p-values
p_values_cat = {}

# Perform Chi-squared test on each categorical column
for col in cat_col_eval:
    p_val = chi2_test(col, hotel_bookings, data_profiling_before)
    p_values_cat[col] = p_val

# Filter columns with p-value less than the threshold
significant_cats = {col: p for col, p in p_values_cat.items() if p < p_value_threshold}

# Prepare data for DataFrame
significant_data_cat = [(col, significant_cats[col]) for col in significant_cats]

# Convert to DataFrame with columns for feature and p-value
significant_cats_df = pd.DataFrame(significant_data_cat, columns=['Feature', 'P-value'])

# Check if significant_cats_df is not empty and print
if not significant_cats_df.empty:
    print('Alert! We found significant differences in the categorical features of the new data!')
    print(significant_cats_df)
    message_chi2 = significant_cats_df.to_html(classes='my-table', index=False, border=0)
    
    # For each significant feature, print the distribution before and after
    for feature in significant_cats_df['Feature']:
        print(f"\nDistribution for {feature} BEFORE:")
        print(hotel_bookings[feature].value_counts(normalize=True).sort_index())
        
        print(f"\nDistribution for {feature} AFTER:")
        print(data_profiling_before[feature].value_counts(normalize=True).sort_index())
else:
    print(f"No significant differences in categorical features found at the specified {p_value_threshold} threshold.")
    message_chi2 = f"No significant differences in categorical features found at the specified {p_value_threshold} threshold."


New Data Exploration for Categorical Columns
CHI^2 TEST
New Data Exploration
Alert! We found significant differences in the categorical features of the new data!
              Feature   P-value
0  reserved_room_type  0.008154

Distribution for reserved_room_type BEFORE:
A    0.720990
B    0.009393
C    0.007676
D    0.160737
E    0.054575
F    0.024186
G    0.017382
H    0.005010
L    0.000051
Name: reserved_room_type, dtype: float64

Distribution for reserved_room_type AFTER:
A    0.532258
C    0.032258
D    0.306452
E    0.048387
F    0.032258
G    0.048387
Name: reserved_room_type, dtype: float64


### SHOW IN FLASK

In [17]:
empty = " "

In [74]:
import json

# Assuming analysis_results is your list of dictionaries with titles and messages
analysis_results = [
    {'title': message_correlation_analysis, 'message': empty},
    {'title': title_hotel_bookings_corr, 'message': message_hotel_bookings_corr},
    {'title':title_data_profiling_before, 'message': message_data_profiling_before},
    {'title':title_count_nulls, 'message': message_count_nulls},
    {'title':title_count_cat, 'message': message_categorical},
    {'title':title_count_num, 'message': message_numerical},
    {'title':title_count_bin, 'message': message_binary},
    {'title': title_outliers, 'message': merged_outliers_message},
    {'title': title_ttest, 'message': message_ttest},
    {'title': title_kstest, 'message': message_kstest},
    {'title':title_chi2, 'message':message_chi2}
 ]

# Save to a JSON file
with open('analysis_results.json', 'w') as outfile:
    json.dump(analysis_results, outfile)